In [2]:
%load_ext autoreload
%autoreload 2

In [9]:
import pandas as pd
from pathlib import Path
import json
import locale
from IPython.display import display
from datetime import datetime
import numpy as np

from helpers import renamePostaja, simplifyColumn,lowerSumniki
from scoring_system import ScoringSystem

Step1: DataFrame for each air pollutant
- read each csv
- input into dataframe
- convert into long format

In [4]:
NEVARNE_SNOVI = [("PM10", 1999), ("SO2", 1997), ("NOX", 1997), ("NO2",1997)]
MESECI = ["Jan", "Feb", "Mar", "Apr", "Maj","Jun", "Jul", "Avg", "Sep", "Okt", "Nov","Dec"]

YEAR_MONTH_SORT = {
    'by': ['Leto', 'Mesec'],
    'key': lambda col: col if col.name != 'Mesec' else col.map(lambda x: datetime.strptime(x, '%b').month)
}

In [5]:
def extend_dataframe(repo: dict[str, pd.DataFrame], snov: str, to_add: pd.DataFrame):
    repo[snov] = (
        pd.concat([repo[snov], to_add], ignore_index=True) if snov in repo else to_add
    )


def dict_to_csv(repo: dict[str, pd.DataFrame], form: str):
    for snov, df in repo.items():
        path_dir = Path(f"podatki/df_{form}")

        if Path.exists(path_dir):
            filepath = path_dir / f"{snov}.csv"
            df.to_csv(filepath, index=False)
            print(f"saved: {filepath}")
        else:
            path_dir.mkdir()


def load_air_pollutants(
    nevarne_snovi: list[tuple[str, int]],
) -> dict[str, pd.DataFrame]:
    pd_dict = {}

    for snov, zac_leto in nevarne_snovi:
        dir_path = f"podatki/{snov}"
        file_list = sorted(x for x in Path(dir_path).iterdir())

        for leto, file_path in enumerate(file_list, start=zac_leto):
            df = pd.read_csv(file_path, na_values=["", ",", " ", "/"])

            # cleanup
            df = renamePostaja(df)
            df = simplifyColumn(df, "Postaja", True)

            # insert year column
            df.insert(1, "Leto", leto)
            # wide to long conversion
            df = df.melt(
                id_vars=["Postaja", "Leto"],
                value_vars=MESECI,
                var_name="Mesec",
                value_name=snov,
            )

            # concat the dataframe
            extend_dataframe(pd_dict, snov, df)

    return pd_dict


postaje_dict = load_air_pollutants(NEVARNE_SNOVI)
dict_to_csv(postaje_dict, "postaje")

saved: podatki/df_postaje/PM10.csv
saved: podatki/df_postaje/SO2.csv
saved: podatki/df_postaje/NOX.csv
saved: podatki/df_postaje/NO2.csv


Step2: Map stations to regions

In [6]:
# returns (postaja->regija map, missing postaja)
def postaja_regija_map(slovar_path: str) -> pd.Series:
    # load dict from file
    regija_dict = json.load(open(slovar_path))
    # create dict for postaja->regija
    regija_dict = {
        postaja: regija
        for regija, postaje in regija_dict.items()
        for postaja in postaje
    }
    # create series for mapper; cleanup names, drop duplicates
    regija_series = pd.Series(regija_dict, name="Regija")
    regija_series.index = regija_series.index.map(lowerSumniki)
    regija_series = regija_series[~regija_series.index.duplicated()]
    return regija_series


# check and print if postaja exists without mapping
def check_missing_mapping(postaje_dict: dict[str, pd.DataFrame], map_regija: pd.Series):
    prevodi = set(map_regija.index.unique())
    for snov, df in postaje_dict.items():
        postaje = set(df["Postaja"].unique())
        inter = postaje - prevodi
        if len(inter) > 0:
            print(f"{snov}: {inter}")


map_regija = postaja_regija_map("podatki/slovar.json")
check_missing_mapping(postaje_dict, map_regija)

In [7]:
def add_regije_df(
    postaje_dict: dict[str, pd.DataFrame], mapper: pd.Series
) -> dict[str, pd.DataFrame]:
    regije_dict = {}
    for snov, df in postaje_dict.items():
        regija_df = df.copy()
        regija_df.insert(loc=1, column="Regija", value=regija_df["Postaja"].map(mapper))

        regije_dict[snov] = regija_df

    return regije_dict

#Step3 parameter za izracun
regije_dict = add_regije_df(postaje_dict=postaje_dict, mapper=map_regija)
dict_to_csv(regije_dict,"regije")

saved: podatki/df_regije/PM10.csv
saved: podatki/df_regije/SO2.csv
saved: podatki/df_regije/NOX.csv
saved: podatki/df_regije/NO2.csv


In [30]:
locale.setlocale(locale.LC_ALL, "sl_SI.utf8")
for snov, df in regije_dict.items():
    df2 = df.copy().drop(columns=["Postaja"])

    scoringSystem = ScoringSystem(df2, snov)

    best_range, regions, score = scoringSystem.optimal_year_range()
    kart = scoringSystem.full_df
    total_score = scoringSystem.calculate_total_score()
    total_score.to_csv(f"tmp_scoring_{snov}.csv")
    

    print(f"----{snov}----")
    print("Število praznih vnosov:", kart[kart[snov].isna()].size)
    print("najboljši obseg let: ", best_range)

    print("vključene regije:", regions)
    print("točke v obsegu:", score)

----PM10----
Število praznih vnosov: 3824
najboljši obseg let:  (2006, 2023)
vključene regije: ['Goriška', 'Jugovzhodna Slovenija', 'Obalno-kraška', 'Osrednjeslovenska', 'Podravska', 'Pomurska', 'Posavska', 'Savinjska', 'Zasavska']
točke v obsegu: 2380.0547088791845
----SO2----
Število praznih vnosov: 6108
najboljši obseg let:  (1997, 2009)
vključene regije: ['Koroška', 'Osrednjeslovenska', 'Podravska', 'Posavska', 'Savinjska', 'Zasavska']
točke v obsegu: 1166.9086878453043
----NOX----
Število praznih vnosov: 3984
najboljši obseg let:  (2008, 2023)
vključene regije: ['Goriška', 'Obalno-kraška', 'Osrednjeslovenska', 'Podravska', 'Savinjska', 'Zasavska']
točke v obsegu: 1485.884090909091
----NO2----
Število praznih vnosov: 3428
najboljši obseg let:  (2008, 2023)
vključene regije: ['Goriška', 'Obalno-kraška', 'Osrednjeslovenska', 'Podravska', 'Savinjska', 'Zasavska']
točke v obsegu: 1483.85118018018
